# 2030 분류 모델

## 2030데이터와 언론사 수집 데이터 합치기

- Train data
    - 4.7 보궐선거 데이터
- Test data
    - 대선을 위해 수집한 데이터

In [1]:
import pandas as pd
import numpy as np
import re
# 형태소 분류 태그
from konlpy.tag import Okt

In [2]:
# 2030 데이터라고 가정한 데이터
data2030 = pd.read_csv('data/data_2030_1.csv', index_col=0)

# 2030 아닌 데이터
data = pd.read_csv('data/재보궐선거댓글데이터_최종_유튜브수정_0429.csv')

In [3]:
# 댓글을 기반으로 다른 언론사에서 수집한 데이터는 2030 연령이 아니라고 가정
data.rename(columns={'댓글':'Comment'}, inplace=True )
data['Comment']

0                           철수야! 뜸 들이지 말고 애국하는 마음으로 물러서라~~~
1         박영선은 정동영이 얻은 36프로선에 머무를것. 4.7.이후 OOO정권은 몰락의 길 ...
2                             빵선이가서울시장되면서울은공산국가수도제2의평양이될것이다
3         서울시장후보더듬당박빵선이는절대로서울시장을할수없다이유는가족은미국.영국에 영주권자이므로...
4         부산은오거돈선거이고 오거돈치부선거아닌가 오거돈에 성추해으로 생긴선거가 가독도신공항은...
                                ...                        
144134    국민의힘 찍지 말라고 이 뉴스가 나온거임\r\n내냔에 국민의힘 찍을라 했드만 망했다...
144135    굳이 일본과 해저터널 해야되는 이유가 없은이유\r\n-일본은고속도로비가 비싸다\r\...
144136           도랏구나. 열도는 걍 갈라파고스로 남겨 둬라. 재난 난민 넘어 오면 귀찮다.
144137                            이걸 왜 하지? 우리한테 아무런 의미가 없는데
144138    우리에겐 아무런 도움이 되지 않는 것을 일본에게는 엄청난 기회가 되는 것을 왜 굳이...
Name: Comment, Length: 144139, dtype: object

In [4]:
# 댓글을 기반으로 2030 키워드로 유튜브에서 찾아낸 데이터는 2030 데이터라고 가정
data2030['Comment']

0                내리막이 있으면 다시 오르막이 있는 법, 예방주사라 생각하고 힘들 냅시다.
1        2030마음을 얻으려면 조언을 2030을 모셔다가 들어보는 성의를 보여야 됩니다. ...
2        전 군대 전역 전까지는 보수를 지지했지만 대학에서 양극화에 관심을 가지면서 진보로 ...
3        몸 안에서 무엇인가 무너져 내린 기분..그래도 힘내고 가야죠민주당은 뼈아픈 진단 새...
4                                         네 죄송합니다. 삭제하겠습니다
                               ...                        
48635              간교한 인간성을 가진,  김종인, 안띨수를 섬기느니..  죽는게 났다.
48636               이준석 정치연륜이 뭐가있다고 이렇게 나와서 이야기를 많이 하고 다니나
48637    준석아 주댕이 조심해라.안철수가 할말이 옳다. 단일화 못햇우면 오세훈 승라없엇다. ...
48638                                           김준석  아웃 밉상
48639                    토론자가 없어서 양문석, 이준석을 부르냐???한심하네~!!!
Name: Comment, Length: 56829, dtype: object

In [5]:
# 새로운 데이터 프레임 만들기
df = pd.concat( [pd.DataFrame(data['Comment']), pd.DataFrame(data2030['Comment'])], axis=0 )

# 2030이 아니면 0, 2030이면 1으로 라벨링
df['2030'] = 0
df.iloc[144139:]['2030'] = 1 # 유튜브에서 수집한 2030 키워드 데이터는 2030 세대라고 가정
df

C:\Users\master4\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Comment,2030
0,철수야! 뜸 들이지 말고 애국하는 마음으로 물러서라~~~,0
1,박영선은 정동영이 얻은 36프로선에 머무를것. 4.7.이후 OOO정권은 몰락의 길 ...,0
2,빵선이가서울시장되면서울은공산국가수도제2의평양이될것이다,0
3,서울시장후보더듬당박빵선이는절대로서울시장을할수없다이유는가족은미국.영국에 영주권자이므로...,0
4,부산은오거돈선거이고 오거돈치부선거아닌가 오거돈에 성추해으로 생긴선거가 가독도신공항은...,0
...,...,...
48635,"간교한 인간성을 가진, 김종인, 안띨수를 섬기느니.. 죽는게 났다.",1
48636,이준석 정치연륜이 뭐가있다고 이렇게 나와서 이야기를 많이 하고 다니나,1
48637,준석아 주댕이 조심해라.안철수가 할말이 옳다. 단일화 못햇우면 오세훈 승라없엇다. ...,1
48638,김준석 아웃 밉상,1


## 데이터 전처리

In [6]:
# 정규식
def text_cleaning(text) :
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', text)
    return result

In [7]:
# 댓글 데이터 한글 정규화 과정
df['Comment'] = df['Comment'].apply( lambda x: text_cleaning(x))

In [8]:
# 형태소 분리 tagger
def get_pos(x) :
    tagger = Okt() # Okt로 형태소 분리하기
    pos = tagger.pos(x) 
    results = [] # 형태소를 담을 리스트
    for i in pos:
        if i[1] != 'Josa': # 조사는 빼버리기
            results.append(f'{i[0]}/{i[1]}') #'단어/품사'의 형태로 리스트에 추가
        else:
            pass
    return results

In [ ]:
# 단어당 몇개씩 들어 있는지에 대한 벡터를 생성
from sklearn.feature_extraction.text import CountVectorizer

index_vectorizer = CountVectorizer(tokenizer= lambda x : get_pos(x))
X = index_vectorizer.fit_transform(df['Comment'].tolist())

## X-y구분